In [ ]:
import requests  # 서버에 http 프로토콜로 요청
from bs4 import BeautifulSoup as bs  # html 파싱
import time  # 5초간 휴식 할 때 사용
from datetime import datetime  # 오늘 연월일 추출
import pandas as pd  # 데이터 프레임 만듬
from tqdm import tqdm
import sqlite3
from sqlalchemy import create_engine
import yes24_dbio as ydb

In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import time
from selenium.common.exceptions import NoSuchElementException

In [ ]:
data = pd.read_csv('./data/yes24_08-23월간베스트.csv')

In [ ]:
colnames = ['yy', 'mon', 'image', 'category', 'title', 'title_sub', 'link', 'author', \
         'publisher', 'pub_date', 'price', 'n_reviews', 'detail_link', 'rating', 'tags']

In [ ]:
data.columns = colnames

In [ ]:
data

In [ ]:
review_page = []

for links in data["link"]:

    # driver는 해상도에 맞춰서 빈 사이트를 띄워주는 로직 
    try:
        driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
        driver.set_window_size(1920, 1080) # 웹 브라우저 해상도 조절
        driver.get(links) # for 루프 안에 있는 link를 하나씩 넣으면서 페이지를 띄우게함 

        time.sleep(2)# 2초 쉬었다가 
        down_button = driver.find_element(By.CSS_SELECTOR,"#yDetailTabNavWrap > div > div.gd_tabBar > ul > li:nth-child(2) > a")
        down_button.click()
        # 댓글을 수집하기 위해서 배송/반품/교환 섹션으로 내려가게끔 페이지 다운 버튼 클릭 
        time.sleep(2)
        review_btn = driver.find_element(By.CSS_SELECTOR, '#infoset_reviewContentList > div:nth-child(3) > div.reviewInfoBot.crop > a > div')
        review_btn.click()
        time.sleep(3)
        # 도착한 포커스 내에서 페이지를 모두 저장 시킴(html_page라는 변수에)
        html_page = driver.page_source
        # 그 수집한 html_page 변수를 detail_page라는 리스트에 저장 시킴 
        detail_page.append(html_page)
        # 그 이후에 페이지 변수를 증가시킴 
        page = 1 
        nth_child = 4
        time.sleep(3)
        while True:
            print(f"페이지 : {page}, nth-child : {nth_child}, 페이지 개수 {len(detail_page)}")
            time.sleep(2)
            next_btn = driver.find_element(By.CSS_SELECTOR,f"#infoset_reviewContentList > div.review_sort.sortTop > div.review_sortLft > div > a:nth-child({nth_child})")
            next_btn.click()
            time.sleep(2)
            nth_child += 1
            page +=1 
            html_page = driver.page_source
            review_page.append(html_page)
            time.sleep(3)
            try: 
                empty_button = driver.find_element(By.CSS_SELECTOR,"#infoset_reviewContentList > div.review_sort.sortTop > div.review_sortLft > div > a.bgYUI.next")
                break
            except NoSuchElementException:
                pass

            if page % 5 == 0:
                time.sleep(2)
                next_prev_button = driver.find_element(By.CSS_SELECTOR,"#infoset_reviewContentList > div.review_sort.sortTop > div.review_sortLft > div > a.bgYUI.next")
                next_prev_button.click()
                page += 1 
                nth_child = 4
                html_page = driver.page_source
                review_page.append(html_page)
                time.sleep(3)
            
    except Exception :
        print(Exception)
        pass
    
    finally:
        driver.quit()
            
        # DB설계를 하고 해야함..
        # Book_id를 설정해야하는데 어케하는지모름

In [ ]:
review_page